In [170]:
import pandas as pd
import numpy as np
import string
import lxml.etree
import urllib
import urllib.request
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from ast import literal_eval
import matplotlib.pyplot as plt
from textstat.textstat import textstat
from gensim.corpora import wikicorpus
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pandas as pd
import pymongo
from selenium.common.exceptions import NoSuchElementException
from typing import List, Tuple, Dict, Any, Generator, Iterable
import nltk
from collections import defaultdict
import pymongo
from pymongo import MongoClient
import pickle
%matplotlib inline
# Make it pretty
plt.style.use('ggplot')

In [162]:
def browse_to_category(browser, category):
    """ Use browser to get Wikipedia Category page."""
    url = 'https://en.wikipedia.org/wiki/Category:' + category
    browser.get(url)

def get_category_title(browser):
    """ Use css tag 'h1.firstHeading' to find the main category."""
    headline = browser.find_elements_by_css_selector('h1.firstHeading')
    title = [text.text for text in headline]
    return title[0].partition('Category:')[2]

def get_sub_categories(browser):
    """ Use css tag 'a.CategoryTreeLabel' to find the sub-categories."""
    subs = browser.find_elements_by_css_selector('a.CategoryTreeLabel')
    return [category.text for category in subs]

def cick_to_sub_category_page(page):
    """ Click a sub_category and be taken to its pages"""
    browser.find_element_by_link_text(page).click()

def filter_pages(pages):
    for page in pages:
        if len(page) < 2:
            pages.remove(page)
    return pages

def get_wiki_xml(title):
    title = title
    params = { "format":"xml", "action":"query", "prop":"revisions", "rvprop":"timestamp|user|comment|content" }
    params["titles"] = "API|%s" % urllib.parse.quote(title.encode("utf8"))
    qs = "&".join("%s=%s" % (k, v)  for k, v in params.items())
    url = "http://en.wikipedia.org/w/api.php?%s" % qs
    tree = lxml.etree.parse(urllib.request.urlopen(url))
    revs = tree.xpath('//rev')
    return (revs[-1].text)

def clean_wiki_markup(raw_article):
    semi_cleaned_article = wikicorpus.filter_wiki(raw_article)
    cleaned_article = semi_cleaned_article.replace("\n", "").replace("\'", "").replace("()", "").replace("=", "").replace("|alt","").replace("\xa0","")
    return cleaned_article
def find_num_categories(raw_article):
    return raw_article.count("[[Category:")
def find_num_images(raw_article):
    return raw_article.count("[[Image:")
def find_num_ISBN(raw_article):
    return raw_article.count("ISBN")
def find_num_references(raw_article):
    return raw_article.count("</ref>")
def find_article_length(cleaned_article):
    return len(cleaned_article)
def find_num_difficult_words(cleaned_article):
    return textstat.difficult_words(cleaned_article)
def find_dale_chall_readability_score(cleaned_article):
    return textstat.dale_chall_readability_score(cleaned_article)
def find_automated_readability_index(cleaned_article):
    return textstat.automated_readability_index(cleaned_article)
def find_linsear_write_formula(cleaned_article):
    return textstat.linsear_write_formula(cleaned_article)
def find_gunning_fog_index(cleaned_article):
    return textstat.gunning_fog(cleaned_article)
def find_syllable_count(cleaned_article):
    return textstat.syllable_count(cleaned_article)
def find_lexicon_count(cleaned_article):
    return textstat.lexicon_count(cleaned_article, removepunct=True)
def find_sentence_count(cleaned_article):
    return textstat.sentence_count(cleaned_article)
def find_smog_index(cleaned_article):
    return textstat.smog_index(cleaned_article)
def find_num_web_citations(raw_article):
    return raw_article.count("{{cite web")
def find_num_book_citations(raw_article):
    return raw_article.count("{{cite book")
def find_num_news_citations(raw_article):
    return raw_article.count("{{cite news")
def find_num_quotes(raw_article):
    return raw_article.count("quote=")
def find_num_h3_headers(raw_article):
    return raw_article.count("\n===")
def find_num_internal_links(raw_article):
    return (raw_article.count("[[") // 2)
def find_num_h2_headers(raw_article):
    return (raw_article.count("\n==") - find_num_h3_headers(raw_article))
def find_num_note_tags(raw_article):
    return raw_article.count("{{note")
def find_num_bullet_points(raw_article):
    return (raw_article.count("*"))
def find_num_underlines(raw_article):
    return (raw_article.count("<u>"))
def find_num_journal_citations(raw_article):
    return (raw_article.count("{{cite journal"))
def find_num_about_links(raw_article):
    return (raw_article.count("{{About"))
def find_num_wikitables(raw_article):
    return (raw_article.count('class="wikitable'))
def find_num_footnotes(raw_article):
    return raw_article.count("{{")
def find_infobox(raw_article):
    return int('{{Infobox' in raw_article)

# Process Start-to-finish:

## 1) Go to wikipedia category page

In [220]:
category = 'Machine_learning'
browser = Chrome()

In [221]:
browse_to_category(browser, category)

## 2) Grab Overarching Category title

In [222]:
Overarching_title = get_category_title(browser)

In [223]:
Overarching_title

'Machine learning'

## 3) Grab Subcategories

In [224]:
subs = get_sub_categories(browser)

## 4) Grab Subcategory information and store in MongoDB

Create MongoDB

In [225]:
client = MongoClient('mongodb://localhost:27017/')
Machine_Learning_db = client['Machine_Learning_db']
collection = Machine_Learning_db['Machine_Learning_db']

Iterate over sub-categories and add their subsquent pages in MongoDB

In [226]:
for sub_cat in subs:
    browse_to_category(browser, category)
    cick_to_sub_category_page(sub_cat)
    title = get_category_title(browser)
    print(title)
    try:
        w = browser.find_element_by_class_name('mw-category')
        pages = filter_pages(w.text.split('\n'))
        for page in pages:
            post = {'category': title,
                'page': page,
                'text' : get_wiki_xml(page)}
            posts = Machine_Learning_db.posts
            post = posts.insert_one(post)
    except NoSuchElementException:
        w = browser.find_elements_by_class_name('mw-content-ltr')
        pages = filter_pages(w[0].text.split('\n'))[2:]
        for page in pages:
            post = {'category': title,
                'page': page,
                'text' : get_wiki_xml(page)}
            posts = Machine_Learning_db.posts
            post = posts.insert_one(post)

Applied machine learning


KeyboardInterrupt: 

## 5) Convert MongoDB to pandas DF and clean/transform data

In [155]:
data = pd.DataFrame(list(posts.find()))

In [157]:
data.drop(columns='_id',inplace=True)

In [160]:
data = data[data['text'] != ""]
data = data[data['text'].str.contains("#redirect") == False]
data = data[data['text'].str.contains("may refer to:\n\n*") == False]
data = data[data['text'].str.contains("can refer to:\n") == False]
data = data[data['text'].str.contains("could refer to:\n") == False]
data = data[data['text'].str.contains("#REDIRECT") == False]
data = data[data['text'].str.contains("== Matches ==\n:") == False]
data = data[data['text'].str.contains("{{underconstruction") == False]

In [163]:
data['cleaned_text'] = data['text'].apply(clean_wiki_markup)
data['num_web_citations'] = data['text'].apply(find_num_web_citations)
data['num_book_citations'] = data['text'].apply(find_num_book_citations)
data['num_news_citations'] = data['text'].apply(find_num_news_citations)
data['num_quotes'] = data['text'].apply(find_num_quotes)
data['num_h3_headers'] = data['text'].apply(find_num_h3_headers)
data['num_internal_links'] = data['text'].apply(find_num_internal_links)
data['num_h2_headers'] = data['text'].apply(find_num_h2_headers)
data['has_infobox'] = data['text'].str.contains('{{Infobox').astype(int)
data['num_categories'] = data['text'].apply(find_num_categories)
data['num_images'] = data['text'].apply(find_num_images)
data['num_ISBN'] = data['text'].apply(find_num_ISBN)
data['num_references'] = data['text'].apply(find_num_references)
data['article_length'] = data['text'].apply(find_article_length)
data['num_difficult_words'] = data['cleaned_text'].apply(find_num_difficult_words)
data['dale_chall_readability_score'] = data['cleaned_text'].apply(find_dale_chall_readability_score)
data['readability_index'] = data['cleaned_text'].apply(find_automated_readability_index)
data['linsear_write_formula'] = data['cleaned_text'].apply(find_linsear_write_formula)
data['gunning_fog_index'] = data['cleaned_text'].apply(find_gunning_fog_index)
data['smog_index'] = data['cleaned_text'].apply(find_smog_index)
data['syllable_count'] = data['cleaned_text'].apply(find_syllable_count)
data['lexicon_count'] = data['cleaned_text'].apply(find_lexicon_count)
data['sentence_count'] = data['cleaned_text'].apply(find_sentence_count)
data['num_footnotes'] = data['text'].apply(find_num_footnotes)
data['num_note_tags'] = data['text'].apply(find_num_note_tags)
data['num_underlines'] = data['text'].apply(find_num_underlines)
data['num_journal_citations'] = data['text'].apply(find_num_journal_citations)
data['num_about_links'] = data['text'].apply(find_num_about_links)
data['num_wikitables'] = data['text'].apply(find_num_wikitables)

Error(DCRS): Word Count is zero cannot divide
Error(ARI) : Sentence count is zero, cannot divide
Error(GF): Word Count is Zero, cannot divide


In [165]:
data.dropna(inplace=True)

## 6) Pickle DataFrame

In [167]:
data.to_pickle('Machine_Learning_Category_Wikipedia.pkl')

## 7) Split data into corrext input (X)

In [168]:
X = data.loc[:, ['has_infobox','num_categories','num_images','num_ISBN','num_references','article_length',
                'num_difficult_words','dale_chall_readability_score','readability_index','linsear_write_formula',
                'gunning_fog_index', 'num_web_citations','num_book_citations','num_news_citations',
                'num_quotes','num_h3_headers','num_internal_links', 'num_h2_headers', 'syllable_count',
                'lexicon_count', 'sentence_count','num_footnotes', 'num_note_tags', 'num_underlines', 'num_journal_citations',
                'num_about_links', 'num_wikitables', 'smog_index']].values

In [171]:
rf_model = pickle.load(open('../src/random_forest_model.sav', 'rb'))

/Users/austin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [173]:
predictions = rf_model.predict(X)

In [174]:
data['Predicted_Quality'] = predictions

In [176]:
Cat_page_df = data.loc[:, ['category', 'page', 'Predicted_Quality']]

In [178]:
Cat_page_df.groupby(by='category').mean()

,Predicted_Quality
category,
Applied machine learning,1.885266
Artificial intelligence conferences,1.528246
Artificial neural networks,1.669305
Bayesian networks,1.622311
Classification algorithms,1.957296
Cluster analysis,1.924664
Computational learning theory,1.587266
Data mining and machine learning software,1.669155
Datasets in machine learning,1.793419


In [186]:
data[data['Predicted_Quality'] >= 4]

,category,page,text,cleaned_text,num_web_citations,num_book_citations,num_news_citations,num_quotes,num_h3_headers,num_internal_links,...,syllable_count,lexicon_count,sentence_count,num_footnotes,num_note_tags,num_underlines,num_journal_citations,num_about_links,num_wikitables,Predicted_Quality
83,Artificial neural networks,Deep learning,{{pp|small=yes}} \n{{About||deep versus shallo...,Deep learning (also known as deep structured ...,23,7,2,0,18,120,...,10332.9,5849,304,211,0,0,37,1,1,4.441802
347,Data mining and machine learning software,Aphelion (software),{{about|the image analysis software suite|astr...,The Aphelion Imaging Software Suite is a softw...,45,0,0,0,7,152,...,7961.4,4324,252,193,0,0,18,0,1,4.232865
365,Data mining and machine learning software,Folding@home,{{Infobox software\n| name = Folding@home...,Folding@home (FAH or F@h) is a distributed com...,126,4,2,0,16,239,...,13167.0,7368,344,229,0,0,70,0,0,4.707155
683,Machine learning task,Speech recognition,{{About||the human linguistic concept|Speech p...,Speech recognition is the inter-disciplinary s...,29,9,5,2,25,156,...,13968.0,7941,374,117,0,0,27,1,0,4.002893
706,Markov models,Kalman filter,{{Use American English|date=March 2018}}\n[[Fi...,The Kalman filter keeps track of the estimated...,4,28,0,2,13,110,...,13138.2,7360,335,108,0,0,37,0,0,4.050635
783,Machine learning researchers,Ray Kurzweil,{{Infobox person\n| name = Ray Kurzwe...,"Raymond Kurzweil ( ; born February 12, 1948) i...",69,6,10,7,15,128,...,10403.1,6236,292,123,0,0,4,0,0,4.110722


In [188]:
data[data['category'] == 'Artificial neural networks']

,category,page,text,cleaned_text,num_web_citations,num_book_citations,num_news_citations,num_quotes,num_h3_headers,num_internal_links,...,syllable_count,lexicon_count,sentence_count,num_footnotes,num_note_tags,num_underlines,num_journal_citations,num_about_links,num_wikitables,Predicted_Quality
46,Artificial neural networks,Artificial neural network,{{Machine learning bar}}\n{{Use dmy dates|date...,An artificial neural network is an interconnec...,8,21,2,1,50,243,...,16399.8,9210,476,287,0,0,81,0,0,3.420194
47,Artificial neural networks,Types of artificial neural networks,There are many '''types of artificial neural n...,There are many types of artificial neural netw...,5,2,0,0,33,73,...,7131.6,4042,224,62,0,0,12,0,0,2.896976
48,Artificial neural networks,Activation function,{{About||the formalism used to approximate the...,Logistic activation functionIn artificial neur...,0,3,0,0,2,40,...,2052.0,1110,69,116,7,0,2,1,2,3.545440
50,Artificial neural networks,Adaptive neuro fuzzy inference system,An '''adaptive neuro-fuzzy inference system'''...,An adaptive neuro-fuzzy inference system or ad...,0,0,0,0,0,3,...,201.6,107,6,8,0,0,1,0,0,0.400464
51,Artificial neural networks,Adaptive resonance theory,'''Adaptive resonance theory''' ('''ART''') is...,Adaptive resonance theory (ART) is a theory de...,0,0,0,0,0,16,...,2139.3,1184,59,4,0,0,0,0,0,2.300937
53,Artificial neural networks,AlterEgo,{{coi|date=April 2018}}\n\n{{Use mdy dates|dat...,AlterEgo is a wearable silent speech output-in...,1,0,0,0,0,6,...,257.4,154,7,5,0,0,0,1,0,0.627825
54,Artificial neural networks,Artificial Intelligence System,{{For|the computer science and machine intelli...,Artificial Intelligence System (AIS) was a dis...,4,0,0,0,0,9,...,656.1,357,23,12,0,0,0,0,0,1.193413
55,Artificial neural networks,Artificial neuron,An '''artificial neuron''' is a [[Function (ma...,An artificial neuron is a mathematical functio...,0,4,0,0,4,39,...,2802.6,1662,88,21,0,0,0,0,0,2.783397
56,Artificial neural networks,Artisto,{{Orphan|date=August 2016}}\n\n{{Infobox softw...,Artisto is a video processing application with...,12,0,0,0,0,11,...,670.5,390,22,23,0,0,3,0,0,1.952647
57,Artificial neural networks,Autoassociative memory,"'''Autoassociative memory''', also known as ''...","Autoassociative memory, also known as auto-ass...",0,0,0,0,5,4,...,446.4,225,14,10,0,0,2,0,0,1.490393


In [189]:
type(browser)

selenium.webdriver.chrome.webdriver.WebDriver

In [192]:
ML_page_predictions = data

In [194]:
ML_page_predictions.to_pickle('ML_page_predictions.pkl')